In [2]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

sys.path.append(r'C:\Users\DerDo\Desktop\fantasy_basketball_project')
from functions import *

Players

In [9]:
soup = get_soup('https://www.basketball-reference.com/players/a/antetgi01.html')

In [80]:
# col_names = [col.get_text() for col in soup.select('th[scope="col"]')]

# for i in ['Highlight', '', 'In Stathead']:
#     col_names.remove(i)

# ids = [i for i, x in enumerate(col_names) if x == 'Season']

# cols = []
# for i, x in enumerate(ids):
#     try: 
#         cols.append(col_names[x: ids[i + 1]])
#     except IndexError as e:
#         cols.append(col_names[x:])

# next_season_projected_cols = cols[0]
# per_gam

per_game_data = soup.select('table[data-soc-sum-table-type="PlayerPerGame"]')

if not per_game_data:
    print('No Data Found')
    # break

### Regular Season Table

per_game_reg_season_columns = per_game_data[0].select('th[scope="col"]')
per_game_reg_season_columns = [
    col.get_text(strip=True) for col in per_game_reg_season_columns
]

per_game_reg_season_table_body = per_game_data[0].select('tbody')[0].select('th,td')
table_data = []
for i in per_game_reg_season_table_body:
    table_data.append(i.get_text(strip=True))

reg_season_data = []
BATCH_SIZE = 31
for i in range(0, len(table_data), BATCH_SIZE):
    curr = table_data[i: i+BATCH_SIZE]
    reg_season_data.append(curr)

### Playoff Table

per_game_playoff_columns = per_game_data[1].select('th[scope="col"]')
per_game_playoff_columns = [
    col.get_text(strip=True) for col in per_game_playoff_columns
]

per_game_playoff_table_body = per_game_data[1].select('tbody')[0].select('th,td')
table_data = []
for i in per_game_playoff_table_body:
    table_data.append(i.get_text(strip=True))

playoff_data = []
BATCH_SIZE = 31
for i in range(0, len(table_data), BATCH_SIZE):
    curr = table_data[i: i+BATCH_SIZE]
    playoff_data.append(curr)

playoff_data

[['2014-15',
  '20',
  'MIL',
  'NBA',
  'SG',
  '6',
  '6',
  '33.5',
  '4.3',
  '11.8',
  '.366',
  '0.0',
  '0.2',
  '.000',
  '4.3',
  '11.7',
  '.371',
  '.366',
  '2.8',
  '3.8',
  '.739',
  '2.0',
  '5.0',
  '7.0',
  '2.7',
  '0.5',
  '1.5',
  '1.8',
  '3.0',
  '11.5',
  ''],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['2016-17',
  '22',
  'MIL',
  'NBA',
  'SF',
  '6',
  '6',
  '40.5',
  '10.0',
  '18.7',
  '.536',
  '0.7',
  '1.7',
  '.400',
  '9.3',
  '17.0',
  '.549',
  '.554',
  '4.2',
  '7.7',
  '.543',
  '1.7',
  '7.8',
  '9.5',
  '4.0',
  '2.2',
  '1.7',
  '3.3',
  '3.5',
  '24.8',
  ''],
 ['2017-18',
  '23',
  'MIL',
  'NBA',
  'PF',
  '7',
  '7',
  '40.0',
  '9.9',
  '17.3',
  '.570',
  '0.6',
  '2.0',
  '.286',
  '9.3',
  '15.3',
  '.607',
  '.587',
  '5.4',
  '7.9',
  '.691',
  '1.1',
  '8.4',
  '9.6',
  '6.3',
  '1.4',
  '0

In [79]:

final_data

[['2013-14',
  '19',
  'MIL',
  'NBA',
  'SF',
  '77',
  '23',
  '24.6',
  '2.2',
  '5.4',
  '.414',
  '0.5',
  '1.5',
  '.347',
  '1.7',
  '3.9',
  '.440',
  '.463',
  '1.8',
  '2.6',
  '.683',
  '1.0',
  '3.4',
  '4.4',
  '1.9',
  '0.8',
  '0.8',
  '1.6',
  '2.2',
  '6.8',
  'ROY-7'],
 ['2014-15',
  '20',
  'MIL',
  'NBA',
  'SG',
  '81',
  '71',
  '31.4',
  '4.7',
  '9.6',
  '.491',
  '0.1',
  '0.5',
  '.159',
  '4.6',
  '9.1',
  '.511',
  '.496',
  '3.2',
  '4.3',
  '.741',
  '1.2',
  '5.5',
  '6.7',
  '2.6',
  '0.9',
  '1.0',
  '2.1',
  '3.1',
  '12.7',
  ''],
 ['2015-16',
  '21',
  'MIL',
  'NBA',
  'PG',
  '80',
  '79',
  '35.3',
  '6.4',
  '12.7',
  '.506',
  '0.4',
  '1.4',
  '.257',
  '6.1',
  '11.3',
  '.537',
  '.520',
  '3.7',
  '5.1',
  '.724',
  '1.4',
  '6.2',
  '7.7',
  '4.3',
  '1.2',
  '1.4',
  '2.6',
  '3.2',
  '16.9',
  ''],
 ['2016-17',
  '22',
  'MIL',
  'NBA',
  'SF',
  '80',
  '80',
  '35.6',
  '8.2',
  '15.7',
  '.521',
  '0.6',
  '2.3',
  '.272',
  '7.6',
  '